Test Modèle

## Part 7 - Test modèle - Régression logistique

In [2]:
pip install -U scikit-learn    

Requirement already up-to-date: scikit-learn in c:\python39\lib\site-packages (1.1.2)
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import pip 
pip.main(["install","matplotlib"])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


You should consider upgrading via the 'c:\Python39\python.exe -m pip install --upgrade pip' command.


0

In [4]:
!pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in c:\python39\lib\site-packages (1.1.2)


You should consider upgrading via the 'c:\python39\python.exe -m pip install --upgrade pip' command.


In [5]:
# Chargement des librairies nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, RocCurveDisplay, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_curve
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) # to avoid deprecation warnings

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

In [6]:
# Chargement de la dataset
dataset = pd.read_csv("dataset_CV_labelise.csv", delimiter = ";", encoding = "ISO-8859-1")

In [7]:
dataset.head()
dataset.shape

(4049, 10)

In [8]:
# Basic stats
print("Number of rows : {}".format(dataset.shape[0]))
print()

print("Display of dataset: ")
display(dataset.head())
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Number of rows : 4049

Display of dataset: 


,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens,%texte_lu,%texte_lu_fin_ligne,Is_alpha,Grammar,Label
0,SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,SELMA LAFKIR CORDE 80 CODEUSE ENTHOUSIASTE,CV_1,0.0,6.0,2.80,97.20,"[True, True, True, False, True, True]","['NOUN', 'PROPN', 'VERB', 'NUM', 'PROPN', 'PRO...",1
1,PROFIL,PROFIL,CV_1,1.0,1.0,3.27,96.73,[True],['NOUN'],0
2,PERSONNEL Je suis Ã©tudiante au lycÃ©e Condorcet.,PERSONNEL Je suis Ã©tudiante au lycÃ©e Condorcet.,CV_1,2.0,8.0,7.01,92.99,"[True, True, True, True, True, True, True, False]","['NOUN', 'PRON', 'AUX', 'ADJ', 'ADP', 'NOUN', ...",0
3,Je code depuis l'Ã¢ge de 13 ans et j'aime crÃ©...,Je code depuis l'Ã¢ge de 13 ans et j'aime crÃ©...,CV_1,3.0,24.0,18.22,81.78,"[True, True, True, False, True, True, False, T...","['PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', ...",0
4,RÃALISATIONS,RÃALISATIONS,CV_1,4.0,1.0,18.69,81.31,[True],['NOUN'],0



Basics statistics: 


,CV_Sentences,Sentences_CV_clean,CV_Number,Sentence_line,Nb_tokens,%texte_lu,%texte_lu_fin_ligne,Is_alpha,Grammar,Label
count,4049,4045,4047,4047.000000,4047.000000,4047.000000,4047.000000,4047,4047,4049.000000
unique,2829,2825,145,NaN,NaN,NaN,NaN,1251,2058,NaN
top,#NOM?,#NOM?,CV_10,NaN,NaN,NaN,NaN,[False],['NOUN'],NaN
freq,131,131,66,NaN,NaN,NaN,NaN,463,249,NaN
mean,NaN,NaN,NaN,15.822090,9.097850,50.400032,49.599970,NaN,NaN,0.040257
std,NaN,NaN,NaN,11.372538,10.625067,30.183407,30.183408,NaN,NaN,0.196585
min,NaN,NaN,NaN,0.000000,1.000000,0.240000,0.000000,NaN,NaN,0.000000
25%,NaN,NaN,NaN,7.000000,2.000000,24.310000,23.635000,NaN,NaN,0.000000
50%,NaN,NaN,NaN,14.000000,5.000000,49.760000,50.240000,NaN,NaN,0.000000
75%,NaN,NaN,NaN,23.000000,12.000000,76.365000,75.690000,NaN,NaN,0.000000



Percentage of missing values: 


CV_Sentences           0.000000
Sentences_CV_clean     0.098790
CV_Number              0.049395
Sentence_line          0.049395
Nb_tokens              0.049395
%texte_lu              0.049395
%texte_lu_fin_ligne    0.049395
Is_alpha               0.049395
Grammar                0.049395
Label                  0.000000
dtype: float64

In [9]:
dataset["Label"].unique()
dataset = dataset.loc[(dataset['CV_Sentences'] != "#NOM?"),:]
dataset.shape

(3918, 10)

In [10]:
dataset = dataset.loc[(dataset['CV_Sentences'] != ":"),:]
dataset.shape

(3914, 10)

In [11]:
dataset = dataset.dropna(axis =0, how = 'any')
dataset.shape

(3910, 10)

In [12]:
display(100*dataset.isnull().sum()/dataset.shape[0])

CV_Sentences           0.0
Sentences_CV_clean     0.0
CV_Number              0.0
Sentence_line          0.0
Nb_tokens              0.0
%texte_lu              0.0
%texte_lu_fin_ligne    0.0
Is_alpha               0.0
Grammar                0.0
Label                  0.0
dtype: float64

Multivariate logistic regression

In [25]:
# Separate target variable Y from features X
print("Separating labels from features...")
features_list = ["CV_Sentences","Sentences_CV_clean","CV_Number", "Is_alpha", "Grammar", "Label"]
target_variable = "Label"

X = dataset.drop(features_list, axis = 1)
Y = dataset.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
print(X.head())

Separating labels from features...
...Done.

Y : 
0    1
1    0
2    0
3    0
4    0
Name: Label, dtype: int64

X :
   Sentence_line  Nb_tokens  %texte_lu  %texte_lu_fin_ligne
0            0.0        6.0       2.80                97.20
1            1.0        1.0       3.27                96.73
2            2.0        8.0       7.01                92.99
3            3.0       24.0      18.22                81.78
4            4.0        1.0      18.69                81.31


In [26]:
# Automatically detect positions of numeric/categorical features
idx = 0
numeric_features = []
numeric_indices = []
categorical_features = []
categorical_indices = []
for i,t in X.dtypes.iteritems():
    if ('float' in str(t)) or ('int' in str(t)) :
        numeric_features.append(i)
        numeric_indices.append(idx)
    else :
        categorical_features.append(i)
        categorical_indices.append(idx)

    idx = idx + 1

print('Found numeric features ', numeric_features,' at positions ', numeric_indices)
print('Found categorical features ', categorical_features,' at positions ', categorical_indices)

Found numeric features  ['Sentence_line', 'Nb_tokens', '%texte_lu', '%texte_lu_fin_ligne']  at positions  [0, 1, 2, 3]
Found categorical features  []  at positions  []


In [27]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
# WARNING : don't forget stratify=Y for classification problems
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0, stratify = Y)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [28]:
# Convert pandas DataFrames to numpy arrays before using scikit-learn
print("Convert pandas DataFrames to numpy arrays...")
X_train = X_train.values
X_test = X_test.values
Y_train = Y_train.tolist()
Y_test = Y_test.tolist()
print("...Done")

print(X_train[0:5,:])
print(X_test[0:2,:])
print()
print(Y_train[0:5])
print(Y_test[0:2])

Convert pandas DataFrames to numpy arrays...
...Done
[[ 7.    6.   67.72 32.28]
 [ 6.    1.   17.58 82.42]
 [12.   34.   95.47  4.53]
 [11.   28.   50.62 49.38]
 [ 1.    1.    0.49 99.51]]
[[15.   13.   82.58 17.42]
 [ 1.   33.   13.55 86.45]]

[0, 0, 0, 0, 1]
[0, 0]


Training pipeline

In [29]:
# Missing values
print("Imputing missing values...")
print(X_train[0:5,:])
print()
imputer = SimpleImputer(strategy="mean")
X_train[:,numeric_indices] = imputer.fit_transform(X_train[:,numeric_indices])
print("...Done!")
print(X_train[0:5,:]) 
print() 

# Encoding categorical features and standardizing numerical features
print("Encoding categorical features and standardizing numerical features...")
print()
print(X_train[0:5,:])

# Normalization
numeric_transformer = StandardScaler()

# OHE / dummyfication
categorical_transformer = OneHotEncoder(drop='first')

featureencoder = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_indices),    
        ('num', numeric_transformer, numeric_indices)
        ]
    )

X_train = featureencoder.fit_transform(X_train)
print("...Done")
print(X_train[0:5,:])

# Label encoding
print("Encoding labels...")
print(Y_train[0:5])
encoder = LabelEncoder()
Y_train = encoder.fit_transform(Y_train)
print("...Done")
print(Y_train[0:5])

Imputing missing values...
[[ 7.    6.   67.72 32.28]
 [ 6.    1.   17.58 82.42]
 [12.   34.   95.47  4.53]
 [11.   28.   50.62 49.38]
 [ 1.    1.    0.49 99.51]]

...Done!
[[ 7.    6.   67.72 32.28]
 [ 6.    1.   17.58 82.42]
 [12.   34.   95.47  4.53]
 [11.   28.   50.62 49.38]
 [ 1.    1.    0.49 99.51]]

Encoding categorical features and standardizing numerical features...

[[ 7.    6.   67.72 32.28]
 [ 6.    1.   17.58 82.42]
 [12.   34.   95.47  4.53]
 [11.   28.   50.62 49.38]
 [ 1.    1.    0.49 99.51]]
...Done
[[-0.77628051 -0.30248416  0.58084389 -0.58084396]
 [-0.86533341 -0.76821091 -1.08793027  1.08793009]
 [-0.33101601  2.30558561  1.50442752 -1.50442752]
 [-0.42006891  1.74671352  0.01171669 -0.01171679]
 [-1.31059791 -0.76821091 -1.65672466  1.65672443]]
Encoding labels...
[0, 0, 0, 0, 1]
...Done
[0 0 0 0 1]


In [30]:
# Train model
print("Train model...")
classifier = LogisticRegression()
classifier.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


In [31]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = classifier.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

Predictions on training set...
...Done.
[0 0 0 ... 0 0 0]



Test pipeline

In [32]:
# Missing values
print("Imputing missing values...")
print(X_test[0:5,:])
X_test[:,numeric_indices] = imputer.transform(X_test[:,numeric_indices])
print("...Done!")
print(X_test[0:5,:]) 
print() 

# Encoding categorical features and standardizing numerical features
print("Encoding categorical features and standardizing numerical features...")
print(X_test[0:5,:])
X_test = featureencoder.transform(X_test)
print("...Done")
print(X_test[0:5,:])

# Label encoding
print("Encoding labels...")
print(Y_test[0:5])
Y_test = encoder.transform(Y_test)
print("...Done")
print(Y_test[0:5])

Imputing missing values...
[[15.   13.   82.58 17.42]
 [ 1.   33.   13.55 86.45]
 [ 2.    8.   19.3  80.7 ]
 [22.    1.   87.91 12.09]
 [ 9.    2.   37.16 62.84]]
...Done!
[[15.   13.   82.58 17.42]
 [ 1.   33.   13.55 86.45]
 [ 2.    8.   19.3  80.7 ]
 [22.    1.   87.91 12.09]
 [ 9.    2.   37.16 62.84]]

Encoding categorical features and standardizing numerical features...
[[15.   13.   82.58 17.42]
 [ 1.   33.   13.55 86.45]
 [ 2.    8.   19.3  80.7 ]
 [22.    1.   87.91 12.09]
 [ 9.    2.   37.16 62.84]]
...Done
[[-0.06385731  0.34953328  1.07541877 -1.0754188 ]
 [-1.31059791  2.21244026 -1.22205791  1.22205772]
 [-1.22154501 -0.11619346 -1.03068473  1.03068455]
 [ 0.55951299 -0.76821091  1.25281339 -1.25281341]
 [-0.59817471 -0.67506556 -0.43626298  0.43626284]]
Encoding labels...
[0, 0, 1, 0, 0]
...Done
[0 0 1 0 0]


In [33]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = classifier.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

Predictions on test set...
...Done.
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [34]:
# Print scores
print("accuracy on training set : ", accuracy_score(Y_train, Y_train_pred))
print("accuracy on test set : ", accuracy_score(Y_test, Y_test_pred))
print()

print("f1-score on training set : ", f1_score(Y_train, Y_train_pred))                                            ## ATTENTION REVOIR F1-SCORE
print("f1-score on test set : ", f1_score(Y_test, Y_test_pred))
print()

accuracy on training set :  0.9584398976982097
accuracy on test set :  0.9578005115089514

f1-score on training set :  0.0
f1-score on test set :  0.0



In [35]:
# Visualize ROC curves
probas_train = classifier.predict_proba(X_train)[:,1]
precisions, recalls, thresholds = roc_curve(Y_train, probas_train)
fig = go.Figure(
    data = go.Scatter(
        name = 'train',
        x = recalls, 
        y = precisions, 
        mode = 'lines'
    ),
    layout = go.Layout(
        title = go.layout.Title(text = "ROC curve", x = 0.5),
        xaxis = go.layout.XAxis(title = 'False Positive Rate'),
        yaxis = go.layout.YAxis(title = 'True Positive Rate')
    )
)

probas_test = classifier.predict_proba(X_test)[:,1]
precisions, recalls, thresholds = roc_curve(Y_test, probas_test)
fig.add_trace(go.Scatter(
    name = 'test',
    x = recalls, 
    y = precisions, 
    mode = 'lines'
    )
)
fig.show()